# Data Loading Helper Classes and Types

> This notebook contains helper classes and types for loading data from media datasets.

In [ ]:
#| default_exp utils.data_loading

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations

import xarray as xr
import pandas as pd
import numpy as np

from typing import (
    Dict, Union, Literal, 
    Optional, Set, Tuple, 
    List, Callable)
import warnings
import os
import json
from pathlib import Path


In [ ]:
#| export
class DataLoader:
    """Load data from a file or a directory of files, while keeping track of the metadata."""
    def __init__(
        self, 
        path: Union[str, Path], # path to the file or directory
        metadata_file_name: Optional[str] = None, # name of the metadata file
        custom_data_loader: Optional[Union[Callable, str]] = None # custom data loader function or name of module with custom data loader
        ) -> DataLoader:
        ...

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()